In [31]:
import os 

In [32]:
pwd


'C:\\Users\\syed PC\\Documents\\chest-cancer-project\\end-to-end-chest-cancer-project'

In [1]:
cd "C:\Users\syed PC\Documents\chest-cancer-project\end-to-end-chest-cancer-project"


C:\Users\syed PC\Documents\chest-cancer-project\end-to-end-chest-cancer-project


In [2]:
pwd

'C:\\Users\\syed PC\\Documents\\chest-cancer-project\\end-to-end-chest-cancer-project'

In [33]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mohdsyed/end-to-end-chest-cancer-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mohdsyed"
os.environ["MLFLOW_TRACKING_PASSWORD"]="c40da0aafe921184bc8d4042bead5ead6ae01cf9" 



In [34]:
import tensorflow as tf 

In [35]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int



In [37]:
from cnnClassifier.constants import *

from cnnClassifier.utils.common import read_yaml, create_directories, save_json



In [38]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/mohdsyed/end-to-end-chest-cancer-project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [39]:


import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse



In [40]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")




In [42]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e



[2025-03-12 19:45:48,193: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-12 19:45:48,201: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-12 19:45:48,205: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 15s 2s/step - loss: 76.7743 - accuracy: 0.5686
[2025-03-12 19:46:03,908: INFO: common: json file saved at: scores.json]


2025/03/12 19:46:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-03-12 19:46:06,590: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\SYEDPC~1\AppData\Local\Temp\tmp6jeubj1g\model\data\model\assets
[2025-03-12 19:46:08,342: INFO: builder_impl: Assets written to: C:\Users\SYEDPC~1\AppData\Local\Temp\tmp6jeubj1g\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/03/12 19:47:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
